# Ramblings: Rejoinder a Regularización y Expansiones Basales


## Diferencia entre MSE, RMSE, MAE

- _Median Absolute Error_ $\leadsto$ Magnitud promedio de los errores, agnóstica a la dirección.

$$
\textsf{MAE} = \frac{1}{n}\sum_{j=1}^{n}\vert y_{j} - \hat{y}_{j} \vert
$$

- _Median Squared Error y Root Median Squared Error_: Regla cuadrática que evalúa la función de pérdida en un plano de coordenadas

$$
\textsf{MSE} = \frac{1}{n}\sum_{j=1}^{n}(y_{j} - \hat{y}_{j})^{2}
$$

$$
\textsf{RMSE} = \sqrt{\textsf{MSE}}
$$

MAE y RMSE expresan el error promedio en términos del vector objetivo, con un rango de $\varepsilon\rightarrow [0, \infty^{+})$. Ambos son agnósticos a la dirección de los errores (RMSE preserva cardinalidad y evita suma cero mediante $\sqrt{X}$ y MAE preserva cardinalidad mediante valor absoluto)

MSE permite diagnosticar si las predicciones realizadas por el modelo subestiman o sobreestiman el comportamiento, dado su signo.

#### Manual de Cortapalos en funciones de pérdida contínuas

- Si MAE $>$ (R)MSE, significa que el punto equidistante de los errores es superior a la métrica promedio cuadrático. En esta situación, esperamos tener observaciones atípicas con __sobreestimaciones__. Esta situación puede ocurrir cuando nuestro modelo falla en identificar outliers. Esperamos tener una mayor concentración en valores bajos de la distribución del error

- Si MAE $<$ (R)MSE, significa que el punto equidistante de los errores es superior a la métrica promedio cuadrático. En esta situación, esperamos tener observaciones atípicas con __subestimaciones__. Esta situación puede ocurrir cuando neustro modelo falla en identificar outliers. Esperamos tener una mayor concentración en valores altos de la distribución del error.

- MSE permite reportar la sub/sobre estimación del modelo __respecto a la esperanza matemática del vector objetivo__. $\mathbb{E}[y]$.


## GLMNET: Modelos generalizados regularizados 

Los métodos vistos con `sklearn` por defecto implementan MCO. Se puede flexibilizar la familia estocástica a regularizar mediante GLMNET.

https://web.stanford.edu/~hastie/glmnet_python/

## Profundización en las variantes CV


In [1]:
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_regression

In [2]:
sim_data = make_regression(n_samples=1000,
                           n_features=100,
                           n_targets=1)

In [3]:
std_attr = StandardScaler().fit_transform(sim_data[0])

X_train, X_test, y_train, y_test = train_test_split(std_attr,
                                                   sim_data[1])

* Hiperparámetros relevantes del modelo:
    - alphas: Rango de valores del contorno regularizador ($\lambda$)
    - cv: Cantidad de validaciones cruzadas.
    - scoring:  https://scikit-learn.org/stable/modules/model_evaluation.html
    
    
* Podemos definir el rango de $\lambda$ y la cantidad de regularizadores

In [4]:
ridge_cv = RidgeCV(alphas = [0.001, 0.01, 0.5, 1],
                   cv=10,
                   scoring="neg_mean_absolute_error").fit(X_train, y_train)

* También podemos extraer la métrica específica en cada valor del hiperparámetro y una cantidad "óptima" de validaciones cruzadas inferidas mediante LOO.

In [5]:
ridge_cv_auto_loo = RidgeCV(alphas = [0.001, 0.01, 0.5, 1],
                           store_cv_values=True).fit(X_train, y_train)

#### Ver especificación de hiperparámetros

In [6]:
ridge_cv.set_params()

{'alphas': array([0.001, 0.01 , 0.5  , 1.   ]),
 'cv': 10,
 'fit_intercept': True,
 'gcv_mode': None,
 'normalize': False,
 'scoring': 'neg_mean_absolute_error',
 'store_cv_values': False}

Podemos pedir el score (por defecto da $r^2$), puaj :(

In [12]:
ridge_cv.score(X_test, y_test)

0.99999999999693

### Elemento importante a considerar en un objeto creado con `sklearn`

Aquellos elementos dentro del objeto que terminan con `_` representan a las instancias del objeto. Con ello podemos extraer información relevante sobre la ejecución del modelo

In [ ]:
ridge_cv.

In [7]:
ridge_cv.alpha_

0.001

In [8]:
ridge_cv.coef_[:10]

array([ 9.34402944e+01, -6.14154263e-06,  7.28066217e+00,  9.85499087e-06,
       -5.58041870e-06,  8.93682686e-06, -1.53987795e-05,  6.22425674e-07,
        1.01074851e-05,  1.38449345e-05])

* Instancias sin un underscore al final, representan parámetros ingresados en la __inicialización__ del método.

In [9]:
ridge_cv.alphas

array([0.001, 0.01 , 0.5  , 1.   ])

* Por último, están los métodos dentro del objeto entrenado, en este caso debemos pasar argumentos :)

In [10]:
ridge_cv.predict(X_test[:10])

array([-173.39166293,  191.44541724,  111.54307564,   18.55440881,
       -115.86652988,   41.30496874,  205.83730278,  -33.33534837,
        268.9704177 , -141.99077754])

* De manera adicional, cuando especificamos `store_cv_value=True` y `cv=None`, podemos extraer el valor de la métrica en toda la grilla

In [11]:
ridge_cv_auto_loo.cv_values_

array([[1.71315962e-08, 1.71306757e-06, 4.27016336e-03, 1.70297903e-02],
       [2.82091427e-07, 2.82081183e-05, 7.03810820e-02, 2.80957925e-01],
       [8.65685595e-09, 8.65658434e-07, 2.16045368e-03, 8.62677435e-03],
       ...,
       [1.74033229e-09, 1.74012355e-07, 4.32200571e-04, 1.71733820e-03],
       [4.90908921e-07, 4.90889872e-05, 1.22463636e-01, 4.88801779e-01],
       [6.04220415e-09, 6.04179124e-07, 1.50484101e-03, 5.99659007e-03]])

## Sobre las funciones de identidad en GAM

* Retomemos el punto de Generalized Linear Models (jerga de Dobson, 2008; King, 1992):
    - Componente estocástico: Proceso de Generación de Datos subyacente.
    - Componente sistemático: Combinación lineal de parámetros.
    - Función de enlace: Generalización de $g(\mathbb{E}[y]) = \mathbf{X}\vec{\beta}$
    
* Las funciones de enlace en el contexto de GLM tenían formas restringidas $\leadsto$ $\textsf{logit}(\mu), \textsf{log}(\mu), 1/\mu$.

* En GAM, la función de identidad toma una forma aproximadamente "no paramétrica", tomando en consideración la distribución conjunta de los datos.

* El método para esto se llama _Backfitting_ $\leadsto$ "Asumamos que $\boldsymbol\beta$ es no identificable, promediemos las observaciones en una ventana de Parzen, repitamos esto hasta que el valor de $\boldsymbol\beta$ se estabilice"

## Dependencia parcial

> Notación de Hastie et al. (2009)

* Objetivo: identificar la naturaleza de la dependencia de nuestra función de identidad $f(X)$ en los valores conjuntos.
    - Limitante: Funciona cuando nuestras dimensiones son bajas :(
    
* La dependencia parcial obtiene un subconjunto de atributos indexados en $\mathcal{S} \subset \{1, 2, \cdots, p\}$.
* Si una función general mapea $f(X)=f(X_{s}, X_{c})$, el __promedio de la dependencia parcial en el subconjunto de atributos será:__

$$
f_{\mathcal{S}}(X_{\mathcal{S}}) = \mathbb{E}_{X_{c}}(X_{\mathcal{S}}, X_{C})
$$

Los métodos de dependencia parcial permiten evaluar algoritmos "de caja negra" mediante:

$$
\bar{f}_{\mathcal{S}}(X_{\mathcal{S}}) = \frac{1}{N}\sum_{i=1}^{N}f(X_{\mathcal{S}}, x_{iC})
$$

De esta manera, los gráficos permite evaluar el efecto específico de una variable __después de considerar el efecto del resto de atributos controlado en las medias__